In [6]:
import numpy as np
import random
import os
import argparse
import time
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from main.layout import Layout
from main.algorithms import enum_layout_wo_rdt, init_S, coarse_to_fined
from main.auto_models import MTSeqBackbone, MTSeqModel
from main.trainer import Trainer

from data.nyuv2_dataloader_adashare import NYU_v2
from data.taskonomy_dataloader_adashare import Taskonomy
from data.pixel2pixel_loss import NYUCriterions, TaskonomyCriterions
from data.pixel2pixel_metrics import NYUMetrics, TaskonomyMetrics

In [3]:
dataroot = '/mnt/nfs/work1/huiguan/lijunzhang/policymtl/data/Taskonomy/'

tasks = ['segment_semantic','normal','depth_zbuffer','keypoints2d','edge_texture']
cls_num = {'segment_semantic': 17, 'normal': 3, 'depth_zbuffer': 1, 'keypoints2d': 1, 'edge_texture': 1}

dataset = Taskonomy(dataroot, 'train', crop_h=224, crop_w=224)
trainDataloader = DataLoader(dataset, batch_size=16, shuffle=True)

dataset = Taskonomy(dataroot, 'test_small', crop_h=224, crop_w=224)
valDataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [4]:
prototxt = 'models/deeplab_resnet34_adashare.prototxt'
coarse_B = 5
mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}

In [5]:
with torch.cuda.device(0):
    backbone = MTSeqBackbone(prototxt)
    fined_B = len(backbone.basic_blocks)
    feature_dim = backbone(torch.rand(1,3,224,224)).shape[1]

In [18]:
T = len(tasks)
    
layout_list = [] 
S0 = init_S(T, coarse_B) # initial state
L = Layout(T, coarse_B, S0) # initial layout
layout_list.append(L)

enum_layout_wo_rdt(L, layout_list)

Coarse Layout:
[[{1, 2, 4}, {0, 3}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}]]
Fined Layout:
[[{1, 2, 4}, {0, 3}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {1, 2}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}], [{0, 3}, {4}, {2}, {1}]]


In [67]:
layout_idx = 4
layout = layout_list[layout_idx]
print('Coarse Layout:', flush=True)
print(layout, flush=True)

layout = coarse_to_fined(layout, fined_B, mapping)
print('Fined Layout:', flush=True)
print(layout, flush=True)

Coarse Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]
Fined Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]


In [68]:
model = MTSeqModel(prototxt, layout=layout, feature_dim=feature_dim, cls_num=cls_num).cuda()

Construct MTSeqModel from Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]


In [69]:
times = []
count = 0
for i, data in enumerate(valDataloader):
    if count < 50:
        count += 1
        x = data['input'].cuda()
        
        start_time = time.time()
        output = model(x)
        end_time = time.time()
        temp = end_time-start_time
    else:
        break
    times.append(temp)

In [70]:
from statistics import mean
mean(times[0:])

0.06403481960296631

In [71]:
times

[0.09326720237731934,
 0.07489919662475586,
 0.0750741958618164,
 0.06736922264099121,
 0.06133151054382324,
 0.06647610664367676,
 0.06582045555114746,
 0.06191563606262207,
 0.05357003211975098,
 0.05968213081359863,
 0.060109615325927734,
 0.06007719039916992,
 0.08457493782043457,
 0.07651734352111816,
 0.06776213645935059,
 0.07460236549377441,
 0.06839561462402344,
 0.07155728340148926,
 0.07279086112976074,
 0.06758785247802734,
 0.06671619415283203,
 0.08404994010925293,
 0.05644869804382324,
 0.06981825828552246,
 0.06050586700439453,
 0.053292274475097656,
 0.06160688400268555,
 0.05935359001159668,
 0.05948328971862793,
 0.053438425064086914,
 0.053099632263183594,
 0.055268049240112305,
 0.05561637878417969,
 0.059213876724243164,
 0.05927014350891113,
 0.059645891189575195,
 0.053897857666015625,
 0.05971193313598633,
 0.060923099517822266,
 0.053665876388549805,
 0.05488109588623047,
 0.08211565017700195,
 0.05456733703613281,
 0.056473493576049805,
 0.06439757347106934,
